In [38]:
import numpy as np
import torch  

In [48]:
def phi(t, x, p , theta) :
    return 0

def argmaxH(t,x,p,q):
    return 0

def b(t, u ,x):
    return -x/4 + u

def sigma(t, u, x):
    return x/5 + u

def h(x):
    return (np.dot(Q@x, x))/2
    
def f(t, x, u):
    return (np.dot(x,x)/2 + np.dot(2*u , u))/2
    
def Hx(t, x , u , p , q):
    return -p/4 + q/5 -x/2 

def argmaxH(x,p,q): 
    return -np.dot(x/2,x)/2 + np.dot(p,-x/4) + np.dot(q, x/5) + np.dot(p+q , p+q)/4


In [49]:
max_steps = 100
n = 25
M = 64
T = 0.1
Q = np.eye(n)
p0 = -0.9586 
x0 = 1
d = 1

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomNeuralNetwork(nn.Module):
    def __init__(self, n, d):
        super(CustomNeuralNetwork, self).__init__()
        
        input_dim = 1 + n + n
        hidden_dim = 10 + n + n
        output_dim = n * d
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, t, x, p):
        print(f"t shape: {t.shape}")  # Vérifier la taille de t
        print(f"x shape: {x.shape}")  # Vérifier la taille de x
        print(f"p shape: {p.shape}")  # Vérifier la taille de p

        input_tensor = torch.cat((t.unsqueeze(1), x, p), dim=-1)  # Correction ici

        print(f"input_tensor shape: {input_tensor.shape}")  # Vérifier la taille de l'entrée finale

        x = F.relu(self.fc1(input_tensor))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)




In [59]:
import torch
import numpy as np



# Initialisation
t = np.arange(0, T, step)
x = np.zeros((M, n+1))
p = np.zeros((M, n+1))
q = np.zeros(n)
u = np.zeros(n)
p[:, 0] = p0  # p0 doit être défini quelque part

# Initialisation du réseau et de l'optimiseur
phi = CustomNeuralNetwork(n, d)
optimizer = torch.optim.Adam(phi.parameters(), lr=nu)
criterion = torch.nn.MSELoss()

for l in range(max_steps):
    x[:, 0] = x0  # x0 doit être défini

    for i in range(n):
        t_i = torch.tensor(t[i], dtype=torch.float32)
        x_i = torch.tensor(x[:, i], dtype=torch.float32)
        p_i = torch.tensor(p[:, i], dtype=torch.float32)

        q[i] = phi(t_i, x_i, p_i).detach().numpy()
        u[i] = argmaxH(t[i], x[i], p[i], q[i])

        for o in range(M):
            x[o, i+1] = x[o, i] + b(t[i], x[o, i], u[i]) * step + sigma(t[i], x[o, i], u[i]) * np.sqrt(step) * np.random.normal()
            p[o, i+1] = p[o, i] - Hx(t[i], x[o, i], u[i], p[o, i], q[i]) * step + q[i] * np.sqrt(step) * np.random.normal()

    # Calcul du coût J
    J = np.mean([np.sum(f(t, x[k], u)) * T/n + h(x[k, -1]) for k in range(M)])

    # Calcul de la loss
    h_x_final = h_x(torch.tensor(x[:, -1], dtype=torch.float32))
    p_final = torch.tensor(p[:, -1], dtype=torch.float32)
    loss = criterion(h_x_final + p_final, torch.zeros_like(h_x_final))

    # Mise à jour des paramètres du modèle
    optimizer.zero_grad()  # Réinitialisation des gradients
    loss.backward()  # Calcul des gradients
    optimizer.step()  # Mise à jour des paramètres du réseau (theta)

t shape: torch.Size([])
x shape: torch.Size([64])
p shape: torch.Size([64])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [44]:
#Version Primitive du code en haut, marche pas 
"""
t = np.arange(0,T,n)
x = np.array([[0 for _ in range(n+1)] for _ in range(M)])
p = np.array([[0 for _ in range(n+1)] for _ in range(M)])
q = np.array([0 for _ in range(n)])
u = np.array([0 for _ in range(n)])
p[:,0] = [p0 for _ in range(M)]
phi = CustomNeuralNetwork(n, d)
for l in range(max_steps):
    x[:,0] = [x0 for _ in range(M)]
    
    for i in range(n):
        q[i] = phi(t[i] , x[i] , p[i] , theta)
        u[i] = argmaxH(t[i], x[i] , p[i] , q[i])
        for o in range(M):
            x[i+1][o] = x[i] + b(t[i], x[i], u[i])*step + sigma(t[i], x[i], u[i])*sqrt(step)*random.normal()
            p[i+1][o] = p[i] - Hx(t[i], x[i], u[i], p[i], q[i])*step + q[i]*sqrt(step)*random.normal()
    J=0
    for k in range(M):
        J+=np.sum(f(t,x[k], u))*T/n + h(x[k][-1])
    J/= M
    loss = np.sum((np.square(h_x(x[:,-1]) + p[:,-1])))/M
    theta , p[0] = theta - nu * gradient_loss

#returns 4-tuple processes (x, u, p , q)_t
"""""
        
    
    
    

'\nt = np.arange(0,T,n)\nx = np.array([[0 for _ in range(n+1)] for _ in range(M)])\np = np.array([[0 for _ in range(n+1)] for _ in range(M)])\nq = np.array([0 for _ in range(n)])\nu = np.array([0 for _ in range(n)])\np[:,0] = [p0 for _ in range(M)]\nphi = CustomNeuralNetwork(n, d)\nfor l in range(max_steps):\n    x[:,0] = [x0 for _ in range(M)]\n    \n    for i in range(n):\n        q[i] = phi(t[i] , x[i] , p[i] , theta)\n        u[i] = argmaxH(t[i], x[i] , p[i] , q[i])\n        for o in range(M):\n            x[i+1][o] = x[i] + b(t[i], x[i], u[i])*step + sigma(t[i], x[i], u[i])*sqrt(step)*random.normal()\n            p[i+1][o] = p[i] - Hx(t[i], x[i], u[i], p[i], q[i])*step + q[i]*sqrt(step)*random.normal()\n    J=0\n    for k in range(M):\n        J+=np.sum(f(t,x[k], u))*T/n + h(x[k][-1])\n    J/= M\n    loss = np.sum((np.square(h_x(x[:,-1]) + p[:,-1])))/M\n    theta , p[0] = theta - nu * gradient_loss\n\n#returns 4-tuple processes (x, u, p , q)_t\n'